## Conversational Retrieval Chain
https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html

In [9]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
import inspect

/home/limcheekin/ws/py/langchain/flutter-gpt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained("bart-lfqa/")
tokenizer = AutoTokenizer.from_pretrained("bart-lfqa/")

def get_llm():
    hf_pipeline = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        min_length=32,
        max_length=256,
        temperature=0.0,
        do_sample=False,
        early_stopping=True,
        num_beams=8,
        top_k=None,
        top_p=None,
        eos_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        num_return_sequences=1
    )
    llm = HuggingFacePipeline(pipeline=hf_pipeline)
    return llm

In [11]:
def get_retriever(k:int = 3):
    embeddings = HuggingFaceEmbeddings(
    model_name="flax-sentence-embeddings/all_datasets_v3_mpnet-base")
    faiss_db = FAISS.load_local("faiss.db", embeddings)
    retriever = faiss_db.as_retriever(search_kwargs={"k": k})
    return retriever

In [28]:
from langchain.memory import ConversationBufferMemory
def get_chat_history(memory) -> str:
    return memory

memory = ConversationBufferMemory(memory_key="chat_history")
qa = ConversationalRetrievalChain.from_llm(get_llm(), get_retriever(), memory=memory, get_chat_history=get_chat_history)

In [26]:
print(inspect.getsource(qa._call))

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        question = inputs["question"]
        get_chat_history = self.get_chat_history or _get_chat_history
        chat_history_str = get_chat_history(inputs["chat_history"])

        if chat_history_str:
            new_question = self.question_generator.run(
                question=question, chat_history=chat_history_str
            )
        else:
            new_question = question
        docs = self._get_docs(new_question, inputs)
        new_inputs = inputs.copy()
        new_inputs["question"] = new_question
        new_inputs["chat_history"] = chat_history_str
        answer = self.combine_docs_chain.run(input_documents=docs, **new_inputs)
        if self.return_source_documents:
            return {self.output_key: answer, "source_documents": docs}
        else:
            return {self.output_key: answer}



In [29]:
query = "What is Flutter?"
result = qa({"question": query})
result


{'question': 'What is Flutter?',
 'chat_history': '',
 'answer': 'Flutter is a toolkit that allows you to create beautiful, natively compiled applications for mobile, web, and desktop from a single codebase. It is free and open source, and is used by developers and organizations around the world. It speeds app development and reduces the cost and complexity of app production across platforms.'}

In [30]:
query = "What is the latest version?"
result = qa({"question": query})
result

{'question': 'What is the latest version?',
 'chat_history': 'Human: What is Flutter?\nAI: Flutter is a toolkit that allows you to create beautiful, natively compiled applications for mobile, web, and desktop from a single codebase. It is free and open source, and is used by developers and organizations around the world. It speeds app development and reduces the cost and complexity of app production across platforms.',
 'answer': 'Flutter is a toolkit that allows you to create beautiful, natively compiled applications for mobile, web, and desktop from a single codebase. If you want to use Flutter to build your own app, you can use the Flutter Engine to create your own Flutter app. You can then import Flutter into your Android or iOS app to render a part of your UI in Flutter.'}

In [31]:
query = "Tell me some of it key features."
result = qa({"question": query})
result

{'question': 'Tell me some of it key features.',
 'chat_history': 'Human: What is Flutter?\nAI: Flutter is a toolkit that allows you to create beautiful, natively compiled applications for mobile, web, and desktop from a single codebase. It is free and open source, and is used by developers and organizations around the world. It speeds app development and reduces the cost and complexity of app production across platforms.\nHuman: What is the latest version?\nAI: Flutter is a toolkit that allows you to create beautiful, natively compiled applications for mobile, web, and desktop from a single codebase. If you want to use Flutter to build your own app, you can use the Flutter Engine to create your own Flutter app. You can then import Flutter into your Android or iOS app to render a part of your UI in Flutter.',
 'answer': 'Flutter is a toolkit that allows you to create beautiful, natively compiled applications for mobile, web, and desktop from a single codebase. It is free and open sourc

### Return Source Documents

In [ ]:
qa = ConversationalRetrievalChain.from_llm(get_llm(), get_retriever(), return_source_documents=True)
chat_history = []
query = "What did the president say about Ketanji Brown Jackson"
result = qa({"question": query, "chat_history": chat_history})
result

### ConversationalRetrievalChain with `map_reduce`

In [32]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

In [33]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], output_parser=None, partial_variables={}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:', template_format='f-string', validate_template=True)

In [ ]:
llm = get_llm()
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="stuff")

chain = ConversationalRetrievalChain(
    retriever=get_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    max_tokens_limit=512,
)

chat_history = []
query = "What is Flutter?"
result = chain({"question": query, "chat_history": chat_history})
result

### ConversationalRetrievalChain with Question Answering with sources

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [ ]:
llm = get_llm()
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_with_sources_chain(llm, chain_type="stuff")

chain = ConversationalRetrievalChain(
    retriever=get_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    max_tokens_limit=1024,
)

chat_history = []
query = "What is the latest version of Flutter?"
result = chain({"question": query, "chat_history": chat_history})
result

## Conversation Agent
https://python.langchain.com/en/latest/modules/agents/agents/examples/conversational_agent.html

In [6]:
%pip install python-dotenv google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 5.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 2.1 MB/s eta 0:00:003.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 2.8 MB/s eta 0:00:004.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 2.4 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 2.9 MB/s eta 0:00:004.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 3.3 MB/s eta 0:00:000:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 3.2 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load .env file to environment variables

In [4]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [7]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.agents import initialize_agent

In [8]:
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]